In [2]:
from eval import *
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import *
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import fetch_20newsgroups
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

import pandas as pd
from tqdm import tqdm
from scipy.linalg import orth
import csv
torch.cuda.set_device(7)

In [3]:
taboola_data_dir = "/local/diq/all/StepContent__nocontent_title_desc"
step_lines = "/local/diq/all/StepLines__nocontent_title_desc"

import os
#print(os.listdir(step_lines))

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

for file in os.listdir(step_lines):
    sc_len = file_len("{}/{}".format(taboola_data_dir, file))
    sl_len = file_len("{}/{}".format(step_lines, file))
    #print(sc_len, sl_len)
    assert sc_len == sl_len

In [4]:
taxonomies = set()
#hack for the moment (jupuyter nb in folder)
for file in tqdm(os.listdir(step_lines)):
    with open("{}/{}".format(taboola_data_dir, file), "r") as f:
        data = set(map(lambda json_obj: json_obj.get('first_level_taxonomy'), map(lambda x: json.loads(x), f.readlines())))
        taxonomies= taxonomies.union(data)

print(taxonomies)
mapping = dict([(name, i) for i, name in enumerate(taxonomies)])
print(mapping)

100%|██████████| 114/114 [00:11<00:00,  9.29it/s]

{'travel', 'finance', 'home and garden', 'art and entertainment', 'society', 'business and industrial', 'automotive and vehicles', 'news', 'law, govt and politics', 'food and drink', 'shopping', 'real estate', 'science', 'technology and computing', 'pets', 'sports', 'health and fitness', 'hobbies and interests', 'religion and spirituality', 'family and parenting', 'style and fashion', 'careers', 'education'}
{'travel': 0, 'finance': 1, 'home and garden': 2, 'art and entertainment': 3, 'society': 4, 'business and industrial': 5, 'automotive and vehicles': 6, 'news': 7, 'law, govt and politics': 8, 'food and drink': 9, 'shopping': 10, 'real estate': 11, 'science': 12, 'technology and computing': 13, 'pets': 14, 'sports': 15, 'health and fitness': 16, 'hobbies and interests': 17, 'religion and spirituality': 18, 'family and parenting': 19, 'style and fashion': 20, 'careers': 21, 'education': 22}


TypeError: map() must have at least two arguments.

In [5]:
text = []
labels = []

for file in tqdm(sorted(os.listdir(step_lines))):
    data = pd.read_csv("{}/{}".format(step_lines, file), header=None, sep='\t', quoting=csv.QUOTE_NONE)[2].tolist()
    text+=data
    
    with open("{}/{}".format(taboola_data_dir, file), "r") as f:
        data = map(lambda json_obj: mapping.get(json_obj.get('first_level_taxonomy'), -1), map(lambda x: json.loads(x), f.readlines()))
        labels += list(data)
        
    #print(len(text), len(labels))
    assert(len(text) == len(labels))

print(len(labels))
print(len(text))

test_batch_size=5000
size = len(labels)
print("Loaded dataset {} with total lines: {}".format("TABOOLA", size))

100%|██████████| 114/114 [00:18<00:00,  7.07it/s]

569893
569893
Loaded dataset TABOOLA with total lines: 569893


In [6]:
#load qt model
checkpoint_dir = '/home/jcjessecai/workspace/taboola/quickthoughts/checkpoints'
with open("{}/config.json".format(checkpoint_dir)) as fp:
    CONFIG = json.load(fp)

WV_MODEL = api.load(CONFIG['embedding'])
qt = QuickThoughts(WV_MODEL, hidden_size=CONFIG['hidden_size'])
trained_params = torch.load("{}/checkpoint_latest.pth".format(checkpoint_dir))
qt.load_state_dict(trained_params['state_dict'])
qt = qt.cuda()
qt.eval()
print("Restored successfully from {}".format(checkpoint_dir))

2019-07-19 15:07:06 INFO     loading projection weights from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-19 15:09:09 INFO     loaded (400000, 300) matrix from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Restored successfully from /home/jcjessecai/workspace/taboola/quickthoughts/checkpoints


In [7]:
data = list(map(lambda x: torch.LongTensor(prepare_sequence(x, WV_MODEL.vocab, no_zeros=True)), mapping.keys()))
packed = safe_pack_sequence(data).cuda()
            
vec_categories = qt(packed).cpu().detach().numpy()
print(vec_categories)

[[-0.07658999 -0.09340584  0.01470476 ...  0.01451465  0.03383676
   0.03353591]
 [-0.15760107 -0.01548272 -0.07186207 ... -0.04399835  0.00394282
   0.06472579]
 [ 0.22808631  0.5511722  -0.11740495 ... -0.01527937 -0.00725939
   0.0724398 ]
 ...
 [-0.0219017  -0.15206882 -0.09195305 ... -0.05870578 -0.05670162
   0.05115658]
 [-0.05923571 -0.14442426 -0.23165044 ... -0.0316557   0.02349955
   0.12594706]
 [-0.1402694   0.05879774 -0.08240668 ... -0.04093776 -0.02952008
   0.16207318]]


In [8]:
#encode data
def make_batch(j):
    """Processes one test batch of the test datset"""
    stop_idx = min(size, j+test_batch_size)
    batch_text, batch_labels  = text[j:stop_idx], labels[j:stop_idx]
    data = list(map(lambda x: torch.LongTensor(prepare_sequence(x, WV_MODEL.vocab, no_zeros=True)), batch_text))
    for i in data:
        if len(i) == 0:
            print(i)
            input()
    packed = safe_pack_sequence(data).cuda()
    return qt(packed).cpu().detach().numpy()

feature_list = [make_batch(i) for i in range(0, size, test_batch_size)]
print("Processed {:5d} batches of size {:5d}".format(len(feature_list), test_batch_size))
qt_features = np.concatenate(feature_list)
print("Test feature matrix of shape: {}".format(qt_features.shape))

Processed   114 batches of size  5000
Test feature matrix of shape: (569893, 1000)


In [9]:
taboola_model_dir = "/home/jcjessecai/workspace/taboola_model"
d2v = Doc2Vec.load("{}/{}".format(taboola_model_dir, "gensim_doc2vec.model"))

2019-07-19 15:11:23 INFO     loading Doc2Vec object from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model
2019-07-19 15:11:24 INFO     loading vocabulary recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.vocabulary.* with mmap=None
2019-07-19 15:11:24 INFO     loading trainables recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.trainables.* with mmap=None
2019-07-19 15:11:24 INFO     loading syn1neg from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.trainables.syn1neg.npy with mmap=None
2019-07-19 15:11:25 INFO     loading wv recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.wv.* with mmap=None
2019-07-19 15:11:25 INFO     loading vectors from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.wv.vectors.npy with mmap=None
2019-07-19 15:11:25 INFO     loading docvecs recursively from /home/jcjessecai/workspace/taboola_model/gensim_doc2vec.model.docvecs.* with m

In [10]:
d2v_features = np.vstack([d2v.infer_vector(simple_preprocess(doc)) for doc in text])
print(d2v_features.shape)

(569893, 600)


In [11]:
#first we compare embedding performance by fitting binary classifier on top
from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(d2v_features, labels)
clf = linear_model.SGDClassifier(loss='log', max_iter=1000, n_jobs=20)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on d2v with train acc: {:.2%} test acc: {:.2%}".format(train_acc, test_acc))

X_train, X_test, y_train, y_test = train_test_split(qt_features, labels)
clf = linear_model.SGDClassifier(loss='log', max_iter=1000, n_jobs=20)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on qt with train acc: {:.2%} test acc: {:.2%}".format(train_acc, test_acc))


Fit logistic model on d2v with train acc: 49.60% test acc: 49.21%
Fit logistic model on qt with train acc: 55.59% test acc: 55.06%
